In [190]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

# 뉴스 데이터 로드
news_data = pd.read_csv('./사회_20230707_143938_date4011_4001.csv')  # 뉴스 데이터 파일 경로에 맞게 수정
news_content = news_data['content']

In [191]:
news_content

0       [\n여준성, ‘文케어’ 설계자 김용익 의원 보좌관 출신        문재인 대통령...
1       [\n\n\n\n\n류근혁 차관  \n\n\n\n여준성 비서관 문재인 대통령은 30...
2       [\n청와대 국민청원 게시판에 청원 글 올라와“백신 접종 부작용 호소 끊이지 않아”...
3       [\n[앵커]북한이 새로 개발한 극초음속 미사일을 시험 발사했다고 주장하고 나서면서...
4       [\n기사내용 요약한국생산성본부와 기금출연 협약식객관성·투명성 확보한 기금운영 기대...
                              ...                        
1438    [\n소설가 온유주 인터뷰(상)\n\n\n\n온유주(温又柔) 소설가. 1980년생....
1439    [\n핵심요약[소통과 포용으로 장애공감사회 만들자] '장애인기업지원센터'창업 원하는...
1440    [\n[주장] 설령 제도가 문제라면 보완하는 게 정치... 사회주택이란 우산 속 시...
1441    [\n\n\n\n\n[서울=뉴시스] 권창회 기자 = 황금숙 서울시사회서비스원지부 부...
1442    [\n\n\n\n\n[서울=뉴시스] 권창회 기자 = 서울시사회서비스원지부 조합원들이...
Name: content, Length: 1443, dtype: object

In [192]:
import re
 
def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  


In [193]:
# 댓글들(comments)에서 하나씩 가져와 reply라는 변수에 list로 저장

replies = []
i = 0
for comment in news_content:
  replies.append(text_clean(comment))
  i += 1
  news_data['content'][i-1] = replies[i-1]
print(replies[i-1])

서울뉴시스 권창회 기자  서울시사회서비스원지부 조합원들이 일 오전 서울 중구 서울시청 앞에서 열린 돌봄서비스 공공성 확보와 돌봄노동자 처우 개선을 위한 서울시 대책 마련 촉구 기자회견에서 손피켓을 들고 있다  사진 영상 제보받습니다 공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다 뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부로 보내주시면 적극 반영하겠습니다


In [194]:
news_content

0       여준성 文케어 설계자 김용익 의원 보좌관 출신        문재인 대통령은 일 보건...
1       류근혁 차관  여준성 비서관 문재인 대통령은 일 보건복지부 제차관에 류근혁 대통령비...
2       청와대 국민청원 게시판에 청원 글 올라와백신 접종 부작용 호소 끊이지 않아개인 체질...
3       앵커북한이 새로 개발한 극초음속 미사일을 시험 발사했다고 주장하고 나서면서 우려의 ...
4       기사내용 요약한국생산성본부와 기금출연 협약식객관성투명성 확보한 기금운영 기대서울뉴시...
                              ...                        
1438    소설가 온유주 인터뷰상온유주温又柔 소설가 년생 대만 출신으로 세 살 때 도쿄로 이주...
1439    핵심요약소통과 포용으로 장애공감사회 만들자 장애인기업지원센터창업 원하는 장애인들에게...
1440    주장 설령 제도가 문제라면 보완하는 게 정치 사회주택이란 우산 속 시민은 어쩔 텐가...
1441    서울뉴시스 권창회 기자  황금숙 서울시사회서비스원지부 부지부장이 일 오전 서울 중구...
1442    서울뉴시스 권창회 기자  서울시사회서비스원지부 조합원들이 일 오전 서울 중구 서울시...
Name: content, Length: 1443, dtype: object

In [195]:
kkma = Kkma()

def text2sentences(text):
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)):
        if len(sentences[idx]) <= 10:
            sentences[idx-1] += (' '+sentences[idx])
            sentences[idx] = ''
    return sentences

In [196]:
sentences = text2sentences(news_content[10])

In [197]:
sentences

['여야 언중 법 개정 보류 특위 설치 합의 언론노조기자협회 등 현업단체 환영 윤 호중 더불어 민주당 원내 대표 가운데와 김 기현 국민의 힘 원내 대표 왼쪽 가 일 오후 국회 의장 실에서 언론 중재 법과 관련해 박 병석 국회의 장과 회동을 마친 뒤 기자들의 질문을 받고 있다 김 봉 규 선임기자 더 불어 민주당과 국민의 힘이 일 언론 중재 및 피해 구제 등에 관한 법률언론 중재 법 개정안을 본회의에 상정하는 대신 포괄적 논의를 위한 특위 설치에 합의한 데 대해 언론 현업단체들은 환영의 뜻을 밝혔다 전국언론노동조합 한국기자협회 방송기자 연합회 한국방송 기술인 연합회 한국 피디 연합회 등 곳은 이날 공동성명을 내고 언론 중재법을 넘어 언론개혁의 핵심 의제들을 논의할 특위 설치를 적극 찬성한다고 밝혔다 민주당은 이날 의원총회와 최고위원회 논의를 거쳐 언론 중재법뿐 아니라 정보통신망 법 방송법 신문법 등을 함께 처리할 특위를 설치 하자고 제안했고 국민의 힘이 이에 합의하며 석 달 간에 걸친 언론 중재 법 개정 논란이 일단락됐다 이날 오후에도 국회 앞에서 사회적 합의 기구 설치를 요구하며 언론 중재 법 개정안 처리 반대 회견을 열었던 언론 현업단체들은 이번 특위 설치에 큰 의미를 부여하는 모습이다 이들은 성명에서 언론개혁의 우선순위를 바로잡고 정쟁이 아닌 사회적 숙의와 합의를 거쳐야 한다는 시민사회 학계 법조계 및 언론 현업단체의 요청을 받아들인 것으로 판단한다며 정당 간 협의가 아니라 시민사회 학계 법조계 및 언론 현업단체 등을 포괄해 사실상 사회적 합의 기구로 구성되어야 할 것이라고 요구했다 국민의 힘에 대해 선 그동안 언론 중재 법 강행 처리 반대가 정략적 차원이 아니었다면 특위 활동에 적극 참여해야 할 것이라고 강조했다 언론 중재 법 개정은 피해자 구제 강화가 시급하다는 요구에 따라 시작된 논의지만 징벌적 손해배상제도 도입 등이 언론과 표현의 자유를 위축시킬 수 있다는 우려가 커지며 지난 석 달 간 뜨거운 논란에 휩싸였다 언론 현업단체들은 이번 논란을 통해 언론에

In [198]:
twitter = Twitter()

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # 개행 문자 제거 및 리스트에 추가
        lines = [line.strip() for line in lines]
    return lines

# 파일 경로 설정
file_path = './stopword.txt'

# 파일을 리스트로 변환
stopwords = read_file_to_list(file_path)

# 결과 출력
print(stopwords)

['text', '않다', '되어다', '되다', '하다', '어떻다', '이렇다', '이다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지

/Users/yujin/miniforge3/envs/pandas_dev/lib/python3.10/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [199]:
stopwords.remove('text')

In [200]:
stopwords

['않다',
 '되어다',
 '되다',
 '하다',
 '어떻다',
 '이렇다',
 '이다',
 '어제',
 '매일',
 '아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼

In [201]:
def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                   if noun not in stopwords and len(noun) > 1]))
    return nouns

<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/j6/q03gf6y53_72jb_wzwbp2b140000gn/T/ipykernel_20189/1986530246.py:4: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [202]:
nouns = get_nouns(sentences)

In [203]:
nouns = get_nouns(sentences)
print(sentences[0])
print()
print(nouns[0])

여야 언중 법 개정 보류 특위 설치 합의 언론노조기자협회 등 현업단체 환영 윤 호중 더불어 민주당 원내 대표 가운데와 김 기현 국민의 힘 원내 대표 왼쪽 가 일 오후 국회 의장 실에서 언론 중재 법과 관련해 박 병석 국회의 장과 회동을 마친 뒤 기자들의 질문을 받고 있다 김 봉 규 선임기자 더 불어 민주당과 국민의 힘이 일 언론 중재 및 피해 구제 등에 관한 법률언론 중재 법 개정안을 본회의에 상정하는 대신 포괄적 논의를 위한 특위 설치에 합의한 데 대해 언론 현업단체들은 환영의 뜻을 밝혔다 전국언론노동조합 한국기자협회 방송기자 연합회 한국방송 기술인 연합회 한국 피디 연합회 등 곳은 이날 공동성명을 내고 언론 중재법을 넘어 언론개혁의 핵심 의제들을 논의할 특위 설치를 적극 찬성한다고 밝혔다 민주당은 이날 의원총회와 최고위원회 논의를 거쳐 언론 중재법뿐 아니라 정보통신망 법 방송법 신문법 등을 함께 처리할 특위를 설치 하자고 제안했고 국민의 힘이 이에 합의하며 석 달 간에 걸친 언론 중재 법 개정 논란이 일단락됐다 이날 오후에도 국회 앞에서 사회적 합의 기구 설치를 요구하며 언론 중재 법 개정안 처리 반대 회견을 열었던 언론 현업단체들은 이번 특위 설치에 큰 의미를 부여하는 모습이다 이들은 성명에서 언론개혁의 우선순위를 바로잡고 정쟁이 아닌 사회적 숙의와 합의를 거쳐야 한다는 시민사회 학계 법조계 및 언론 현업단체의 요청을 받아들인 것으로 판단한다며 정당 간 협의가 아니라 시민사회 학계 법조계 및 언론 현업단체 등을 포괄해 사실상 사회적 합의 기구로 구성되어야 할 것이라고 요구했다 국민의 힘에 대해 선 그동안 언론 중재 법 강행 처리 반대가 정략적 차원이 아니었다면 특위 활동에 적극 참여해야 할 것이라고 강조했다 언론 중재 법 개정은 피해자 구제 강화가 시급하다는 요구에 따라 시작된 논의지만 징벌적 손해배상제도 도입 등이 언론과 표현의 자유를 위축시킬 수 있다는 우려가 커지며 지난 석 달 간 뜨거운 논란에 휩싸였다 언론 현업단체들은 이번 논란을 통해 언론에 대

In [204]:
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

In [205]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence
sent_graph = build_sent_graph(nouns)

In [206]:
def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}
words_graph, idx2word = build_words_graph(nouns)

In [207]:
def get_ranks(graph, d=0.85):
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1
        
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A,B)
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [208]:
sent_rank_idx = get_ranks(sent_graph)
sent_rank_idx

{0: 1.0, 1: 1.0}

In [209]:
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k:sent_rank_idx[k], reverse=True)

In [210]:
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k:word_rank_idx[k], reverse=True)

In [211]:
def summarize(sent_num=10):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num]:
        index.append(idx)
        
    index.sort()
    
    for idx in index:
        summary.append(sentences[idx])
        
    for text in summary:
        print(text)
        print('\n')

In [212]:
summarize()

여야 언중 법 개정 보류 특위 설치 합의 언론노조기자협회 등 현업단체 환영 윤 호중 더불어 민주당 원내 대표 가운데와 김 기현 국민의 힘 원내 대표 왼쪽 가 일 오후 국회 의장 실에서 언론 중재 법과 관련해 박 병석 국회의 장과 회동을 마친 뒤 기자들의 질문을 받고 있다 김 봉 규 선임기자 더 불어 민주당과 국민의 힘이 일 언론 중재 및 피해 구제 등에 관한 법률언론 중재 법 개정안을 본회의에 상정하는 대신 포괄적 논의를 위한 특위 설치에 합의한 데 대해 언론 현업단체들은 환영의 뜻을 밝혔다 전국언론노동조합 한국기자협회 방송기자 연합회 한국방송 기술인 연합회 한국 피디 연합회 등 곳은 이날 공동성명을 내고 언론 중재법을 넘어 언론개혁의 핵심 의제들을 논의할 특위 설치를 적극 찬성한다고 밝혔다 민주당은 이날 의원총회와 최고위원회 논의를 거쳐 언론 중재법뿐 아니라 정보통신망 법 방송법 신문법 등을 함께 처리할 특위를 설치 하자고 제안했고 국민의 힘이 이에 합의하며 석 달 간에 걸친 언론 중재 법 개정 논란이 일단락됐다 이날 오후에도 국회 앞에서 사회적 합의 기구 설치를 요구하며 언론 중재 법 개정안 처리 반대 회견을 열었던 언론 현업단체들은 이번 특위 설치에 큰 의미를 부여하는 모습이다 이들은 성명에서 언론개혁의 우선순위를 바로잡고 정쟁이 아닌 사회적 숙의와 합의를 거쳐야 한다는 시민사회 학계 법조계 및 언론 현업단체의 요청을 받아들인 것으로 판단한다며 정당 간 협의가 아니라 시민사회 학계 법조계 및 언론 현업단체 등을 포괄해 사실상 사회적 합의 기구로 구성되어야 할 것이라고 요구했다 국민의 힘에 대해 선 그동안 언론 중재 법 강행 처리 반대가 정략적 차원이 아니었다면 특위 활동에 적극 참여해야 할 것이라고 강조했다 언론 중재 법 개정은 피해자 구제 강화가 시급하다는 요구에 따라 시작된 논의지만 징벌적 손해배상제도 도입 등이 언론과 표현의 자유를 위축시킬 수 있다는 우려가 커지며 지난 석 달 간 뜨거운 논란에 휩싸였다 언론 현업단체들은 이번 논란을 통해 언론에 대

In [214]:
def keywords(word_num=10):
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)
    for idx in index:
        keywords.append(idx2word[idx])
    
    print(keywords)

In [215]:
keywords()

['기구', '단체', '언론', '강화', '국민', '민주당', '상정', '시민사회', '기대', '대신']
